## **0. はじめに**

コンペの概要と注意点を自分用にまとめます

## **1. reference**
* [Detailed API Introduction](https://www.kaggle.com/sohier/detailed-api-introduction)
* [pandas, Matplotlib（mplfinance）でローソク足チャートを作成](https://note.nkmk.me/python-pandas-matplotlib-candlestick-chart/)
* [OHLC Charts in Python](https://plotly.com/python/ohlc-charts/)

## **2. 注意点**

普段のコンペと違い、APIからtest用のデータを取得し、APIを使ってmodelの評価結果を提出する。<br>
ざっくりとしたコードの全体像は、上記のリンクに記載あり。簡単にまとめると、
```
import gresearch_crypto   　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　##<= 今回のコンペ用python library
env = gresearch_crypto.make_env()   　　　　　　　　　　##<= 初期化。

# Training data is in the competition dataset as usual
train_df = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/train.csv', low_memory=False)
tgt_1_model.fit(train_df)　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　#<= いつもどおりmodelを作成。
tgt_2_model.fit(train_df)
iter_test = env.iter_test()　　　　　　　　　　　　　　　　　　　　　　　　　　　##<= テスト用データをapi経由で取得。
for (test_df, sample_prediction_df) in iter_test:
    sample_prediction_df['Target'] = tgt_1_model.predict(test_df) ##<= モデルを通して提出用データ作成
    env.predict(sample_prediction_df)   ##<= 提出！！
```

大体上記の流れになりそう。<br>
外部マシンを使ったモデルを読み込んだり、いろんなモデルを試せるように、<br>
EDAで一つのnotebook、model作成で一つのnotebook、他のnotebookのモデルを読み込んで提出するだけのnotebookの構成が良さそう。[A.I](↑一つのnotebookじゃなくてもよいか要確認。)<br>



## **3. Libraryインストールと読み込み**

mplfinance is usefull for creating candle chart.

In [ ]:
! pip install mplfinance
import gresearch_crypto
import pandas as pd
import numpy as np
import tqdm
import mplfinance as mpf

## **4.1 train data overview**

In [ ]:
import gresearch_crypto
#%%time
## 読み込みに割と時間がかかるので、時間を確認。

## 専用library(API)を使って初期化。一度初期化。
#env = gresearch_crypto.make_env()

## 読み込み
train_df = pd.read_csv('../input/g-research-crypto-forecasting/train.csv', low_memory=False, 
                       dtype={'Asset_ID': 'int8', 'Count': 'int32', 'row_id': 'int32', 'Count': 'int32', 
                              'Open': 'float64', 'High': 'float64', 'Low': 'float64', 'Close': 'float64', 
                              'Volume': 'float64', 'VWAP': 'float64'
                             }
                      )
train_df["timestamp"] = pd.to_datetime(train_df['timestamp'].astype(int), unit='s')
train_df.set_index("timestamp",inplace=True)
train_df.head(3)

In [ ]:
train_df.tail(3)

In [ ]:
train_df.shape

#### 通貨は１４種類？

In [ ]:
train_df["Asset_ID"].unique()

## **4.1.1 Is timestep 60[s]?**

The diff() method does not work for index..

In [ ]:
train_df["tmp_dt"] = train_df.index
train_df[train_df["Asset_ID"] == 0]["tmp_dt"].diff().value_counts()

## **4.2 asset details over view**

weightってなんやねんっ。後で調べる。

In [ ]:
asset_df = pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv')
asset_df

## **4.3 supplemental train over view**

In [ ]:
supplemental_df = pd.read_csv('../input/g-research-crypto-forecasting/supplemental_train.csv')
supplemental_df["timestamp"] = pd.to_datetime(supplemental_df['timestamp'].astype(int), unit='s')
supplemental_df.set_index("timestamp",inplace=True)
supplemental_df

## **5.1 Visualize by plotly**
普段plotlyをよく使うので。<br>
表示範囲を絞れるので、便利。<br>
dfのindexは、datetimeオブジェクトにしておく必要がある。<br>

In [ ]:
train_df = pd.read_csv('../input/g-research-crypto-forecasting/train.csv', low_memory=False, 
                       dtype={'Asset_ID': 'int8', 'Count': 'int32', 'row_id': 'int32', 'Count': 'int32', 
                              'Open': 'float64', 'High': 'float64', 'Low': 'float64', 'Close': 'float64', 
                              'Volume': 'float64', 'VWAP': 'float64'
                             }
                      )

asset_id = 1
tgt_df = train_df[train_df["Asset_ID"] == asset_id]
tgt_df = tgt_df[['Open', 'High', 'Low', 'Close', 'Volume','timestamp']]  ## ohlcvと時間の情報のみ必要。
tgt_df["timestamp"] = pd.to_datetime(tgt_df['timestamp'].astype(int), unit='s')
tgt_df.set_index("timestamp",inplace=True)
tgt_df

In [ ]:
import plotly.graph_objects as go

tgt_df = tgt_df[0:1000]
fig = go.Figure(data=[go.Candlestick(x=tgt_df.index, open=tgt_df['Open'], high=tgt_df['High'], low=tgt_df['Low'], close=tgt_df['Close'])])
fig.show()

## **5.2.1 Resampling "change period" to daily OHLC**

resumple train_df to daily ohlc data

In [ ]:
## Re read original data
asset_id = 1
tgt_df = train_df[train_df["Asset_ID"] == asset_id]
tgt_df = tgt_df[['Open', 'High', 'Low', 'Close', 'Volume','timestamp']]  ## ohlcvと時間の情報のみ必要。
tgt_df["timestamp"] = pd.to_datetime(tgt_df['timestamp'].astype(int), unit='s')
tgt_df.set_index("timestamp",inplace=True)



In [ ]:
## resample
d_ohlcv = {'Open': 'first',
           'High': 'max',
           'Low': 'min',
           'Close': 'last',
           'Volume': 'sum'}
tgt_df_daily = tgt_df.resample('D').agg(d_ohlcv)  #< "D" mean daily. 
tgt_df_daily = tgt_df_daily[0:1000]
fig = go.Figure(data=[go.Candlestick(x=tgt_df_daily.index, open=tgt_df_daily['Open'], high=tgt_df_daily['High'], low=tgt_df_daily['Low'], close=tgt_df_daily['Close'])])
fig.show()

## **5.2.1 Resampling to 15 minutes OHLC**

In [ ]:
d_ohlcv = {'Open': 'first',
           'High': 'max',
           'Low': 'min',
           'Close': 'last',
           'Volume': 'sum'}
tgt_df_daily = tgt_df.resample('15min').agg(d_ohlcv)  #< 15 minutes. 
tgt_df_daily = tgt_df_daily[0:1000]
fig = go.Figure(data=[go.Candlestick(x=tgt_df_daily.index, open=tgt_df_daily['Open'], high=tgt_df_daily['High'], low=tgt_df_daily['Low'], close=tgt_df_daily['Close'])])
fig.show()

## **5.3 mplfinanceを使ってみる**

**　注意 ** <br>
mpfに渡すDFには、ohlcvの情報と、indexは、datetimeオブジェクトにしておく必要がある。<br>
mpf require "OHLCV" columns and index is datetime object.

Unn, no good.

In [ ]:
mpf.plot(tgt_df[:100],type='candle')

## **6. look into test data**

testデータはapi経由で取得する。未来のデータを使って評価するため、将来データを受け取れる仕組みが必要なため。<br>

In [ ]:
env = gresearch_crypto.make_env()
iter_test = env.iter_test()
(test_df, sample_prediction_df) = next(iter_test)
test_df.head()

In [ ]:
asset_id = 1
tgt_df = test_df[test_df["Asset_ID"] == asset_id]
tgt_df["timestamp"] = pd.to_datetime(tgt_df['timestamp'].astype(int), unit='s')
tgt_df.set_index("timestamp",inplace=True)
tgt_df